# Utils

### Create Run-scripts

In [5]:
import os
import numpy as np
import math

rel_distances = [2, 3, 5, 8, 13, 21, 33, 54]
if False:
    from tqdm import trange
    abstract_runscript_content = """#!/bin/tcsh
#
# This is a StarExec runscript for PyRes. See https://www.starexec.org.
#
# To use this, build a StarExec package with "make starexec" and
# upload it to a StarExec cluster - then follow local instructions
# ;-).
#
source /etc/profile.d/modules.csh
setenv MODULEPATH /starexec/sw/modulefiles/Linux:/starexec/sw/modulefiles/Core:/starexec/sw/lmod/lmod/modulefiles/Core
module load python/3.13.0
python --version
which python3.13.0
echo -n "% Problem    : " ; head -2 $1 | tail -1 | sed -e "s/.*  : //"
set ProblemSPC=`grep " SPC " $1 | sed -e "s/.* : //"`
set default_flags="-tifbsVp -nlargest -HPickGiven5"
set rel_flags="-r [rel_distance]"
set final=" "$1
set ecmd="python3 ./pyres_fof.py $default_flags $rel_flags $final"
python3 --version
if ( `expr "$ProblemSPC" : "FOF.*"` || `expr "$ProblemSPC" : "CNF.*"` ) then
	echo "% Command    : " $ecmd
	/home/starexec/bin/GetComputerInfo -p THIS Model CPUModel RAMPerCPU OS | \
        sed -e "s/Computer     /% Computer   /" \
            -e "s/Model        /% Model      /" \
            -e "s/CPUModel     /% CPU        /" \
            -e "s/RAMPerCPU    /% Memory     /" \
            -e "s/OS           /% OS         /"
	echo -n "% CPULimit   : " ; echo "$STAREXEC_CPU_LIMIT"
	echo -n "% DateTime   : " ; date
	echo "% CPUTime    : "
	$ecmd
else
    echo "% SZS status Inappropriate"
endif
    """

    abstract_file_name = "run_scripts/starexec_run_PyRes_rd_[rel_distance]"

    for f in os.listdir("run_scripts/"):
        os.remove(os.path.join("run_scripts/", f))

    X = np.arange(0, 10)
    def get_rel_distances(X, end_of_linear):
        a = 10
        steepness_expo = 1.5
        additive = -(end_of_linear/a)+steepness_expo
        return np.real(np.where(
            X<=(end_of_linear/a),
            a*X,
            a*(np.emath.logn(steepness_expo, X+additive)*(X+additive)-additive)
        )).astype(int).tolist()

    rel_distances = get_rel_distances(X, 50)
    print(rel_distances)

    for x in tqdm(rel_distances):
        def substitute(text): return text.replace("[rel_distance]", f"{x:03d}")

        file_name = substitute(abstract_file_name)
        file_content = substitute(abstract_runscript_content)
        with open(file_name, "w") as file:
            file.write(file_content)

#### Create config_name/config_id dataframe for solver

In [2]:
import pandas as pd
from xml.etree import ElementTree as et
import yaml

# create pd dataframe out of xml
job_xml_path = "1.1.2_get_config.xml"

df_xml = (
    pd.DataFrame.from_records([
        jobpair.attrib for jobpair in
        et.parse(job_xml_path).iterfind(".//JobPair") # Parse xml
    ])
)

# Determine solver name
solver_name = df_xml["solver-name"].unique()[0]
solver_id = df_xml["solver-id"].unique()[0]
config_names_to_ids = (
    df_xml[["config-name", "config-id"]]
    .groupby("config-name")
    .first()
    ["config-id"]
    .astype(int)
    .to_dict()
)

# write data into versioning.yml
with open("../notes/developing/versioning.yml") as f:
    content = yaml.safe_load(f)
content["versions"][solver_name]["starexec_configs"] = config_names_to_ids
content["versions"][solver_name]["starexec_solver_id"] = solver_id
with open("../notes/developing/versioning.yml", "w") as f:
    yaml.dump(content, f)

In [8]:

with open("../notes/developing/versioning.yml") as f:
    content = yaml.safe_load(f)
content

{'rel_distance_configs': {'fibo_4_to_11': [2.0,
   3.0,
   5.0,
   8.0,
   13.0,
   21.0,
   33.0,
   54.0]},
 'starexec_jobs': {5054: {'apt_version': '1.0.0',
   'cpu_clock_timeout': 5,
   'id': 5054,
   'rel_distances': [2.0, 3.0, 5.0, 8.0, 13.0, 21.0, 33.0, 54.0],
   'wall_clock_timeout': 5},
  5055: {'apt_version': '1.0.0',
   'cpu_clock_timeout': 3,
   'id': 5055,
   'rel_distances': [2.0, 3.0, 8.0, 21.0, 54.0],
   'wall_clock_timeout': 3},
  5111: {'apt_version': '1.1.0',
   'cpu_clock_timeout': 5,
   'id': 5111,
   'rel_distances': [2.0, 3.0, 5.0, 8.0, 13.0, 21.0, 33.0, 54.0],
   'wall_clock_timeout': 5},
  5112: {'apt_version': '0.0.0',
   'cpu_clock_timeout': '?',
   'id': 5112,
   'rel_distances': None,
   'wall_clock_timeout': '?'},
  5135: {'apt_version': '0.0.0',
   'cpu_clock_timeout': 5,
   'id': 5135,
   'rel_distances': None,
   'wall_clock_timeout': 10},
  5136: {'apt_version': '1.0.0',
   'cpu_clock_timeout': 5,
   'id': 5136,
   'rel_distances': [2.0, 3.0, 5.0, 8.0,

In [ ]:
import pandas as pd
import yaml

# Specify parameters
solver_name = "1.1.2"
job_name = solver_name + "_rel_10_5"
output_filename = job_name + ".xml"
cpu_timeout=5
wallclock_timeout=10


# Get relevant problem names
df_tptp = pd.read_csv("data/df_tptp.csv", index_col=0)
relevant_problems = df_tptp.loc[
    df_tptp["problem_form"].isin(["FOF", "CNF"]) & (df_tptp["equality"] == "NEQ"),
    ["Problem"]
].reset_index(drop=True).rename({"Problem": "bench-name"}, axis=1)

# Get bench and jobspace info and merge into relevant problems
# df_bench = pd.read_xml("all_problems_job.xml")[["bench-id", "bench-name", "job-space-id", "job-space-path"]]
# df_bench.to_csv("df_bench.csv")
df_bench = pd.read_csv("data/df_bench.csv", index_col=0)
relevant_problems = pd.merge(
    on="bench-name",
    left=relevant_problems,
    right=df_bench,
    how="inner",
)

# Determine configs and solver id
with open("../notes/developing/versioning.yml") as f:
    solver_config = yaml.safe_load(f)["versions"][solver_name]
    solver_id = solver_config["starexec_solver_id"]
    config_name_to_id = solver_config["starexec_configs"]
configs = pd.DataFrame(config_name_to_id.items(), columns=["config-name", "config-id"])

# Get combinations
df_combs = pd.merge(relevant_problems, configs, how="cross")

# Add solver name and config
df_combs["solver-name"] = solver_name
df_combs["solver-id"] = solver_id

prefix = f"""
<?xml version="1.0" encoding="UTF-8"?><tns:Jobs xmlns:tns="https://starexec.ccs.miami.edu/starexec/public/batchJobSchema.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="https://starexec.ccs.miami.edu/starexec/public/batchJobSchema.xsd batchJobSchema.xsd">
<Job name="{job_name}">
<JobAttributes>
<description value="no description"/>
<queue-id value="1"/>
<bench-framework value="runsolver"/>
<start-paused value="false"/>
<cpu-timeout value="{cpu_timeout}"/>
<wallclock-timeout value="{wallclock_timeout}"/>
<seed value="0"/>
<mem-limit value="128.0"/>
<postproc-id value="13"/>
</JobAttributes>
"""
suffix = f"""
</Job>
</tns:Jobs>
"""


def comb_row_to_string(row):
    key_value_strings = sorted([f'{key}="{value}"' for key, value in row.items()])
    return "<JobPair " + " ".join(key_value_strings) + "/>"
xml_job_pairs = df_combs.apply(comb_row_to_string, axis=1)

with open(output_filename, "w") as f:
    f.write((prefix + "\n".join(xml_job_pairs) + suffix).strip())

df_combs

,bench-name,bench-id,job-space-id,job-space-path,config-name,config-id,solver-name,solver-id
0,TOP008-1.p,4193021,85443,jannis_gehring/TPTP v9.0.0/Problems/TOP,PyRes_rd_002,400676,1.1.2,3772
1,TOP008-1.p,4193021,85443,jannis_gehring/TPTP v9.0.0/Problems/TOP,PyRes_rd_003,400679,1.1.2,3772
2,TOP008-1.p,4193021,85443,jannis_gehring/TPTP v9.0.0/Problems/TOP,PyRes_rd_005,400677,1.1.2,3772
3,TOP008-1.p,4193021,85443,jannis_gehring/TPTP v9.0.0/Problems/TOP,PyRes_rd_008,400673,1.1.2,3772
4,TOP008-1.p,4193021,85443,jannis_gehring/TPTP v9.0.0/Problems/TOP,PyRes_rd_013,400675,1.1.2,3772
...,...,...,...,...,...,...,...,...
33323,MED009+1.p,4178902,85416,jannis_gehring/TPTP v9.0.0/Problems/MED,PyRes_rd_008,400673,1.1.2,3772
33324,MED009+1.p,4178902,85416,jannis_gehring/TPTP v9.0.0/Problems/MED,PyRes_rd_013,400675,1.1.2,3772
33325,MED009+1.p,4178902,85416,jannis_gehring/TPTP v9.0.0/Problems/MED,PyRes_rd_021,400680,1.1.2,3772
33326,MED009+1.p,4178902,85416,jannis_gehring/TPTP v9.0.0/Problems/MED,PyRes_rd_033,400678,1.1.2,3772
